In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC ## 🧹 Reporting Factory – Cleanup Temporary Tables
# MAGIC This notebook truncates intermediate tables to save cost and keep your workspace lean.  
# MAGIC It **keeps approved & control data** (risk_rules, risk_eval, report_runs, report_facts).

# COMMAND ----------

# DBTITLE 1,Setup
dbutils.widgets.text("CATALOG", "reporting_factory_risk_profile")
catalog = dbutils.widgets.get("CATALOG")

spark.sql(f"USE CATALOG {catalog}")
print(f"Using catalog: {catalog}")

# COMMAND ----------

# DBTITLE 1,List of tables to truncate
cleanup_targets = [
    "bronze.lending_raw",
    "silver.loans",
    "silver.borrowers",
    "gold.features"
]

# COMMAND ----------

# DBTITLE 1,Truncate the tables (if they exist)
for tbl in cleanup_targets:
    try:
        print(f"⏳ Truncating {tbl} ...")
        spark.sql(f"TRUNCATE TABLE IF EXISTS {tbl}")
        print(f"✅ Truncated {tbl}")
    except Exception as e:
        print(f"⚠️ Skipped {tbl} — {e}")

print("✅ Cleanup complete — temporary tables cleared.")

# COMMAND ----------

# DBTITLE 1,Optional: Drop staging or temp tables
# If your pipeline creates temporary / staging tables, list them here
staging_tables = [
    # Example:
    # "gold.temp_risk_eval",
    # "silver.tmp_borrower_snapshot"
]

for tbl in staging_tables:
    try:
        print(f"🗑 Dropping {tbl} ...")
        spark.sql(f"DROP TABLE IF EXISTS {tbl}")
        print(f"✅ Dropped {tbl}")
    except Exception as e:
        print(f"⚠️ Could not drop {tbl}: {e}")

# COMMAND ----------

# DBTITLE 1,Optional: Compact small Delta files (save cost)
optimize_targets = [
    "gold.risk_eval",
    "gold.report_facts",
    "gold.report_runs"
]

for tbl in optimize_targets:
    try:
        print(f"♻️ Optimizing {tbl} ...")
        spark.sql(f"OPTIMIZE {tbl}")
        print(f"✅ Optimized {tbl}")
    except Exception as e:
        print(f"⚠️ Could not optimize {tbl}: {e}")

print("🏁 Cleanup and compaction complete.")